In [1]:
# libraries
import numpy as np
import pandas as pd 
import torch
from transformers import XLNetConfig, XLNetForTokenClassification, TrainingArguments
import random
from torch.nn.utils.rnn import pad_sequence
from torchmetrics.functional import pearson_corrcoef
from torchmetrics import Metric
from ipynb.fs.full.utils_dh import RegressionTrainer, RiboDatasetGWS, GWSDatasetFromPandas, CorrCoef, collate_fn, compute_metrics, compute_metrics_saved  # custom dataset and trainer

/nfs_home/nallapar/rb-prof/bio_embeds/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-11-08 15:16:00.672066: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-11-08 15:16:03.082812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-08 15:16:03.082942: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-08 15:16:03.082955: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
# reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# dataset paths 
data_folder = '/net/lts2gdk0/mnt/scratch/lts2/nallapar/rb-prof/data/Darnell_Full/Darnell/data_conds_split/processed/'

# model parameters
annot_thresh = 0.5
longZerosThresh_val = 20
percNansThresh_val = 0.05
d_model_val = 256
n_layers_val = 3
n_heads_val = 8
dropout_val = 0.3
lr_val = 1e-4
batch_size_val = 1
alpha = 'NA'

# model name and output folder path
model_name = 'XLNetDepr-' + str(n_layers_val) + '_' + str(d_model_val) + '_' + str(n_heads_val) + '-ALL_NA-PEL-BS' + str(batch_size_val) + '-GWS_PCC_IT192_DH_3LMAEALPHA' + str(alpha) + '_NZ' + str(longZerosThresh_val) + '_PNTh' + str(percNansThresh_val) + '_AnnotThresh' + str(annot_thresh)
output_loc = "saved_models/" + model_name

In [3]:
# generate dataset
ds = 'ALL' # uses all the three conditions
train_dataset, test_dataset = RiboDatasetGWS(data_folder, ds, threshold = annot_thresh, longZerosThresh = longZerosThresh_val, percNansThresh = percNansThresh_val)

# convert pandas dataframes into torch datasets
train_dataset = GWSDatasetFromPandas(train_dataset)
test_dataset = GWSDatasetFromPandas(test_dataset)
print("samples in train dataset: ", len(train_dataset))
print("samples in test dataset: ", len(test_dataset))

samples in train dataset:  6386
samples in test dataset:  1592


In [4]:
# load xlnet to train from scratch
config = XLNetConfig(vocab_size=193, pad_token_id=192, d_model = d_model_val, n_layer = n_layers_val, n_head = n_heads_val, d_inner = d_model_val, num_labels = 1, dropout=dropout_val) # 5^3 + 1 for padding
model = XLNetForTokenClassification(config)

# modify the output layer
model.classifier = torch.nn.Linear(d_model_val, 2, bias=True)

In [5]:
# xlnet training arguments
training_args = TrainingArguments(
    output_dir = output_loc,
    learning_rate = lr_val,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = batch_size_val, # training batch size = per_device_train_batch_size * gradient_accumulation_steps
    per_device_eval_batch_size = 1,
    eval_accumulation_steps = 4, 
    num_train_epochs = 200,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = False,
    dataloader_pin_memory = True,
    save_total_limit = 5,
    dataloader_num_workers = 4,
    include_inputs_for_metrics = True
)

# initialize trainer
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [6]:
# train model
trainer.train()

# save best model
trainer.save_model(output_loc + "/best_model")

/nfs_home/nallapar/rb-prof/bio_embeds/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****


  Num examples = 6386


  Num Epochs = 200


  Instantaneous batch size per device = 1


  Total train batch size (w. parallel, distributed & accumulation) = 1


...



Training completed. Do not forget to share your model on huggingface.co/models =)




Loading best model from saved_models/XLNetDepr-3_256_8-ALL_NA-PEL-BS1-GWS_PCC_IT192_DH_3LMAEALPHANA_NZ20_PNTh0.05_AnnotThresh0.5/checkpoint-408704 (score: 1.1659467220306396).


Saving model checkpoint to saved_models/XLNetDepr-3_256_8-ALL_NA-PEL-BS1-GWS_PCC_IT192_DH_3LMAEALPHANA_NZ20_PNTh0.05_AnnotThresh0.5/best_model


Configuration saved in saved_models/XLNetDepr-3_256_8-ALL_NA-PEL-BS1-GWS_PCC_IT192_DH_3LMAEALPHANA_NZ20_PNTh0.05_AnnotThresh0.5/best_model/config.json


Model weights saved in saved_models/XLNetDepr-3_256_8-ALL_NA-PEL-BS1-GWS_PCC_IT192_DH_3LMAEALPHANA_NZ20_PNTh0.05_AnnotThresh0.5/best_model/pytorch_model.bin


In [7]:
# evaluate model
trainer.evaluate()

***** Running Evaluation *****


  Num examples = 1592


  Batch size = 1


{'eval_loss': 1.1659467220306396,
 'eval_r': 0.5868097543716431,
 'eval_runtime': 26.8424,
 'eval_samples_per_second': 59.309,
 'eval_steps_per_second': 59.309,
 'epoch': 200.0}